# Segmentação de painéis solares utilizando imagens de satélite
---
# Solar Panel Segmentation Using Satellite Imagery

This script is used to visualize the data augmentations.

## Library Imports

In this section, a connection is established with Google Drive files. Ensure that you have a direct access shortcut to the folder named "Segmentacao_de_paineis_solares_utilizando_imagens_de_satelite." Additionally, the necessary Python libraries are imported for the proper functioning of the code.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import numpy as np
import sklearn.model_selection
import sklearn.utils.class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import random
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import json


## Config

In this section, it is possible to define the dataset, the balance between masked and unmasked images, and the file paths to load and save results of data augmentation. Finally, the satellite images and mask of the dataset are unzipped in the Google Colab environment.

In [7]:
## PARAMETER SETTINGS
db = 'google' # dataset; options: google, ign.
opt_databalance = '8020' # (masked/unmasked); options: 9010, 8020, 7030.
batch_size = 8 # maximum 16 due to limitations of free colab.
num_images = 5

## FILE PATHS
origin_dataset_path = "/content/gdrive/MyDrive/Segmentacao_de_paineis_solares_utilizando_imagens_de_satelite/data/raw/bdappv.zip"
dataset_path = "/content/"
image_folder = f'{dataset_path}bdappv/{db}/img'
mask_folder = f'{dataset_path}bdappv/{db}/mask'
black_mask = '/content/gdrive/MyDrive/Segmentacao_de_paineis_solares_utilizando_imagens_de_satelite/data/interim/empty_mask.png'
path_dic_dataset = f'/content/gdrive/MyDrive/Segmentacao_de_paineis_solares_utilizando_imagens_de_satelite/data/interim/jsons/output_google_{opt_databalance}.json'
model_data_augmentation_path = f'/content/gdrive/MyDrive/Segmentacao_de_paineis_solares_utilizando_imagens_de_satelite/assets/data_augmentation.svg'

In [4]:
with zipfile.ZipFile(origin_dataset_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

## Fuctions

In this section, the functions used for loading and plotting data augmentation.

### Loading & Pre-processing

In [18]:
def read_dataset_as_dataframe(path_dic_dataset, name_ds): #path_save_df
    """
    Read a JSON file containing dataset data and create a DataFrame from it.

    Args:
        path_dic_dataset (str): Path to the JSON file containing the dataset data.
        name_ds (str): Name of the dataset in the 'query' key in the JSON file.
        # path_save_df (str): Path to save the resulting DataFrame in CSV format.

    Returns:
        pandas.DataFrame: DataFrame with the randomized dataset data.
    """
    with open(path_dic_dataset, 'r') as file:
        data = json.load(file)
    query = data[name_ds]
    img_files = []
    mask_files = []
    for label in query.keys():
        files_query = query[label][0]
        for file_i in files_query:
            img_files.append(file_i[0])
            mask_files.append(file_i[1])
    df = pd.DataFrame()
    df['imgs'] = img_files
    df['masks'] = mask_files
    df_random = df.sample(frac=1).reset_index(drop=True)
    #df_random.to_csv(f'{path_save_df}df_random_{name_ds}.csv', index=False)
    return df_random

def load_and_preprocess_image(path):
    """
    Loads and preprocesses an image from the given file path.

    Args:
        path (str): File path of the image to be loaded and preprocessed.

    Returns:
        tf.Tensor: Preprocessed image tensor ready for further processing or analysis.
                   The tensor has the shape (height, width, channels) and the pixel values are normalized to the range [0, 1].
    """
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image


def load_and_preprocess_mask(path):
    """
    Loads and preprocesses a mask from the given file path.

    Args:
        path (str): File path of the mask to be loaded and preprocessed.

    Returns:
        tf.Tensor: Preprocessed mask tensor ready for further processing or analysis.
                   The tensor has the shape (height, width, 1) and the pixel values are normalized to the range [0, 1].
    """
    mask = tf.io.read_file(path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    return mask


def data_augmentation(image, mask, size=400, zoom_range=(0.7, 1.2)):
    """
    Applies random data augmentation operations to the given image and mask.
    Random rotation and flip
    Random zoom
    Random brightness, contrast, and saturation adjustments

    Args:
        image (tf.Tensor): Image tensor to be augmented.
                           The tensor has the shape (height, width, channels) and the pixel values are normalized to the range [0, 1].
        mask (tf.Tensor): Mask tensor to be augmented.
                          The tensor has the shape (height, width, 1) and the pixel values are normalized to the range [0, 1].
        size (int): Desired size for the augmented image and mask (both height and width).
        zoom_range (tuple): Range for random zooming. It is specified as a tuple (min_zoom, max_zoom), where values should be between 0 and 1.

    Returns:
        tf.Tensor: Augmented image tensor.
                   The tensor has the shape (size, size, channels) and the pixel values are normalized to the range [0, 1].
        tf.Tensor: Augmented mask tensor.
                   The tensor has the shape (size, size, 1) and the pixel values are normalized to the range [0, 1].
    """
    # Apply random rotation and flip
    combined = tf.concat([image, mask], axis=-1)
    combined = tf.image.random_flip_left_right(combined)
    combined = tf.image.random_flip_up_down(combined)

    rotation_angle = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32) * 90
    combined = tf.image.rot90(combined, k=rotation_angle // 90)

    # Perform random zoom
    zoom_factor = tf.random.uniform([], zoom_range[0], zoom_range[1])
    combined_dims = tf.cast(tf.shape(combined)[:2], tf.float32)
    new_dims = tf.cast(combined_dims * zoom_factor, tf.int32)
    combined_resized = tf.image.resize(combined, new_dims)
    combined = tf.image.resize_with_crop_or_pad(combined_resized, size, size)

    # Split image and mask
    image = combined[:, :, :3]
    mask = combined[:, :, 3:]

    # Apply random brightness, contrast, and saturation adjustments
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.clip_by_value(image, 0, 1)  # Ensure values are in the [0, 1] range

    return image, mask


def random_augmentation(image, mask, augment_prob, size=400, zoom_range=(0.7, 1.2)):
    """
    Applies random data augmentation operations to the given image and mask with a given probability.

    Args:
        image (tf.Tensor): Image tensor to be augmented.
                           The tensor has the shape (height, width, channels) and the pixel values are normalized to the range [0, 1].
        mask (tf.Tensor): Mask tensor to be augmented.
                          The tensor has the shape (height, width, 1) and the pixel values are normalized to the range [0, 1].
        augment_prob (float): Probability of applying data augmentation operations.
                              Should be a value between 0 and 1.
        size (int): Desired size for the augmented image and mask (both height and width).
        zoom_range (tuple): Range for random zooming. It is specified as a tuple (min_zoom, max_zoom), where values should be between 0 and 1.

    Returns:
        tf.Tensor: Augmented image tensor.
                   The tensor has the shape (size, size, channels) and the pixel values are normalized to the range [0, 1].
        tf.Tensor: Augmented mask tensor.
                   The tensor has the shape (size, size, 1) and the pixel values are normalized to the range [0, 1].
    """
    if tf.random.uniform(shape=[], minval=0, maxval=1) < augment_prob:
        return data_augmentation(image, mask, size, zoom_range)
    else:
        return image, mask


def tf_data_generator(image_files, mask_files, batch_size, size=400, zoom_range=(0.7, 1.2)):
    """
    Generates batches of preprocessed image and mask data using TensorFlow's tf.data API.

    Args:
        image_files (list): List of image file paths.
        mask_files (list): List of mask file paths.
        batch_size (int): Batch size.
        size (int): Desired size for resizing the image and mask (both height and width).
        zoom_range (tuple): Range for random zooming. It is specified as a tuple (min_zoom, max_zoom), where values should be between 0 and 1.

    Returns:
        tf.data.Dataset: Dataset containing batches of preprocessed image and mask data.
    """
    # Convert the lists of file paths to tf.data Datasets
    image_dataset = tf.data.Dataset.from_tensor_slices(image_files)
    mask_dataset = tf.data.Dataset.from_tensor_slices(mask_files)

    # Use map to load and preprocess the image and mask data in parallel
    image_dataset = image_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    mask_dataset = mask_dataset.map(load_and_preprocess_mask, num_parallel_calls=tf.data.AUTOTUNE)

    # Combine the image and mask datasets
    dataset = tf.data.Dataset.zip((image_dataset, mask_dataset))

    # Batch the data
    dataset = dataset.batch(batch_size)

    # Prefetch data for improved performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

def tf_data_generator_train(image_files, mask_files, batch_size, num_epochs, augment_prob, size=400, zoom_range=(0.7, 1.2)):
    """
    Generates batches of preprocessed image and mask data using TensorFlow's tf.data API.

    Args:
        image_files (list): List of image file paths.
        mask_files (list): List of mask file paths.
        batch_size (int): Batch size.
        num_epochs (int): Number of epochs.
        augment_prob (float): Probability of applying data augmentation operations.
                              Should be a value between 0 and 1.
        size (int): Desired size for resizing the image and mask (both height and width).
        zoom_range (tuple): Range for random zooming. It is specified as a tuple (min_zoom, max_zoom), where values should be between 0 and 1.

    Returns:
        tf.data.Dataset: Dataset containing batches of preprocessed image and mask data.
    """

    # Convert the lists of file paths to tf.data Datasets
    image_dataset = tf.data.Dataset.from_tensor_slices(image_files)
    mask_dataset = tf.data.Dataset.from_tensor_slices(mask_files)

    # Use map to load and preprocess the image and mask data in parallel
    image_dataset = image_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    mask_dataset = mask_dataset.map(load_and_preprocess_mask, num_parallel_calls=tf.data.AUTOTUNE)

    # Combine the image and mask datasets
    dataset = tf.data.Dataset.zip((image_dataset, mask_dataset))

    # Apply random data augmentation
    dataset = dataset.map(lambda image, mask: random_augmentation(image, mask, augment_prob, size, zoom_range), num_parallel_calls=tf.data.AUTOTUNE)

    # Shuffle the dataset
    dataset = dataset.shuffle(buffer_size=100)

    # Repeat the dataset indefinitely
    dataset = dataset.repeat(num_epochs)

    # Batch the data
    dataset = dataset.batch(batch_size)

    # Prefetch data for improved performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset



### Plots

In [16]:
def display_augmented_images(image_files, mask_files, num_images,model_data_augmentation_path):
    """
    Displays a set of images and their augmented versions side by side.

    Args:
        image_files (list): List of image file paths.
        mask_files (list): List of mask file paths.
        num_images (int): Number of images to display.
    """
    if len(image_files) != len(mask_files):
        raise ValueError("image_files and mask_files must have the same length")

    # Combine image and mask files into pairs
    pairs = list(zip(image_files, mask_files))

    # Randomly select a unique set of pairs (image, mask)
    selected_pairs = random.sample(pairs, num_images)
    i = 0
    for image_path, mask_path in selected_pairs:
        image = load_and_preprocess_image(image_path)
        mask = load_and_preprocess_mask(mask_path)

        augmented_image, augmented_mask = data_augmentation(image, mask)

        # Plot the original image and mask
        fig, ax = plt.subplots(1, 4, figsize=(20, 5))

        ax[0].imshow(image)
        ax[0].set_title("Original Image")
        ax[0].axis('off')

        ax[1].imshow(mask[:,:,0], cmap='gray')
        ax[1].set_title("Original Mask")
        ax[1].axis('off')

        # Plot the augmented image and mask
        ax[2].imshow(augmented_image)
        ax[2].set_title("Augmented Image")
        ax[2].axis('off')

        ax[3].imshow(augmented_mask[:,:,0], cmap='gray')
        ax[3].set_title("Augmented Mask")
        ax[3].axis('off')

        plt.savefig(f'{model_data_augmentation_path[:-4]}_sample{i}{model_data_augmentation_path[-4:]}', bbox_inches='tight', pad_inches=0.02)

        plt.show()
        i = i+1



## Main

The main section of the code contains the read dataset subsection and the visualization of data augmentation.

### Read dataset

In [12]:
df_train = read_dataset_as_dataframe(path_dic_dataset, 'train')
train_img_files = list(df_train['imgs'].values)
train_mask_files = list(df_train['masks'].values)

### Visualize Data Augmentation

In [17]:
display_augmented_images(train_img_files, train_mask_files, num_images,model_data_augmentation_path)

Output hidden; open in https://colab.research.google.com to view.